# 3. Hybrid Model

In [1]:
import pandas as pd
import sys
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)   
# Add project root (one level up from "output/") to Python path
sys.path.append(os.path.abspath(".."))

from models import hybrid_model as Hm
from models.hybrid_model import *


Problem: 
- Name: x1
  Lower bound: 1403760.0
  Upper bound: 1403838.3333333337
  Number of objectives: 1
  Number of constraints: 200
  Number of variables: 486
  Number of binary variables: 0
  Number of integer variables: 480
  Number of continuous variables: 6
  Number of nonzeros: 1382
  Sense: maximize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.032000064849853516
  Error rc: 0
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal_profit= 1430260.0


##### 3.1. Optimal profit value 

In [2]:
unmet_obj=value(model.obj)
'''Since penalty was added to objective thus true profit is given 
by adding deducted penalties'''
optimal_profit=unmet_obj+500*value(sum(u[i,s]for i in range(1,11) 
                                        for s in range(1,6)))
print(f'optval={optimal_profit}')

optval=1430260.0


##### 3.2. Optimal production table

In [3]:
Product = {
    'Product': [f'P{i}' for i in range(1,11)],
    'Factory_1': [value(p[i,1]) for i in range(1,11)],
    'Factory_2': [value(p[i,2]) for i in range(1,11)],
    'Factory_3': [value(p[i,3]) for i in range(1,11)]
}

# Create DataFrame
df_prod = pd.DataFrame(Product)

# Row total (per product across factories)
df_prod["Total_Prod"] = df_prod[['Factory_1','Factory_2','Factory_3']].sum(axis=1)

# Column totals (sum over all products)
df_prod.loc['Total'] = df_prod.drop(columns="Product").sum()
df_prod.loc['Total','Product'] = 'Total'

print("\n**Optimal Production Plan**:\n")
print(df_prod)



**Optimal Production Plan**:

      Product  Factory_1  Factory_2  Factory_3  Total_Prod
0          P1      108.0       83.0       -0.0       191.0
1          P2       -0.0       50.0      156.0       206.0
2          P3       56.0       91.0       26.0       173.0
3          P4       -0.0       65.0      141.0       206.0
4          P5      128.0       70.0       -0.0       198.0
5          P6       -0.0       75.0       96.0       171.0
6          P7      112.0      102.0       -0.0       214.0
7          P8      121.0       93.0       -0.0       214.0
8          P9       -0.0       91.0      116.0       207.0
9         P10       -0.0       80.0       99.0       179.0
Total   Total      525.0      800.0      634.0      1959.0


##### 3.3. Store optimal demand filled and unmet demand tables

In [4]:
Store = {
    'Product': [f'P{i}' for i in range(1,11)],
    'S1': [value(sum(r[i,k,1] for k in range(1,6))) for i in range(1,11)],
    'S2': [value(sum(r[i,k,2] for k in range(1,6))) for i in range(1,11)],
    'S3': [value(sum(r[i,k,3] for k in range(1,6))) for i in range(1,11)],
    'S4': [value(sum(r[i,k,4] for k in range(1,6))) for i in range(1,11)],
    'S5': [value(sum(r[i,k,5] for k in range(1,6))) for i in range(1,11)]
}

# Create DataFrame
df_store = pd.DataFrame(Store)

# Row totals (per product across all stores)
df_store["Total_filled"] = df_store[['S1','S2','S3','S4','S5']].sum(axis=1)

# Column totals (across all products)
df_store.loc['Total'] = df_store.drop(columns="Product").sum()
df_store.loc['Total','Product'] = 'Total'

print("\n**Store Demand Filled**:\n")
print(df_store)

'''Unmet Demands'''

Unmet = {
    'Product': [f'P{i}' for i in range(1,11)],
    'S1': [value(u[i,1]) for i in range(1,11)],
    'S2': [value(u[i,2]) for i in range(1,11)],
    'S3': [value(u[i,3]) for i in range(1,11)],
    'S4': [value(u[i,4]) for i in range(1,11)],
    'S5': [value(u[i,5]) for i in range(1,11)]
}

# Create DataFrame
df_unmet = pd.DataFrame(Unmet)

# Add total unmet demand per product
df_unmet["Total_unmet"] = df_unmet[['S1','S2','S3','S4','S5']].sum(axis=1)

# Add a "Total" row at the bottom
df_unmet.loc['Total'] = df_unmet.drop(columns="Product").sum()
df_unmet.loc['Total','Product'] = 'Total'

print("\n**Store Unmet Demand**:\n")
print(df_unmet)




**Store Demand Filled**:

      Product     S1     S2     S3     S4     S5  Total_filled
0          P1   40.0   38.0   30.0   48.0   35.0         191.0
1          P2   35.0   43.0   45.0   50.0   33.0         206.0
2          P3   45.0   48.0   35.0   43.0    2.0         173.0
3          P4   50.0   35.0   40.0   38.0   43.0         206.0
4          P5   30.0   50.0   48.0   40.0   30.0         198.0
5          P6   25.0   33.0   38.0   45.0   30.0         171.0
6          P7   43.0   45.0   43.0   35.0   48.0         214.0
7          P8   48.0   40.0   33.0   43.0   50.0         214.0
8          P9   38.0   43.0   35.0   48.0   43.0         207.0
9         P10   33.0   30.0   28.0   50.0   38.0         179.0
Total   Total  387.0  405.0  375.0  440.0  352.0        1959.0

**Store Unmet Demand**:

      Product   S1   S2   S3   S4    S5  Total_unmet
0          P1 -0.0 -0.0 -0.0 -0.0  -0.0          0.0
1          P2 -0.0 -0.0 -0.0 -0.0  -0.0          0.0
2          P3 -0.0 -0.0 -0.0 -0.

##### 3.4. Factory to warehouse optimal shipping allocation

In [5]:
Ship_FW = []
for j in range(1,4):      # Factories
    for k in range(1,6):  # Warehouses
        row = {
            'Factory': f'F{j}',
            'Warehouse': f'W{k}'
        }
        # Add shipments for each product
        for i in range(1,11):
            row[f'P{i}'] = value(q[i, j, k])
        Ship_FW.append(row)

df_ship1 = pd.DataFrame(Ship_FW)

# Add row totals (sum across P1...P10)
df_ship1["Total_Shipment"] = df_ship1[[f'P{i}' for i in range(1,11)]].sum(axis=1)

# Add column totals (sum across all factory-warehouse pairs)
totals = df_ship1.drop(columns=["Factory","Warehouse"]).sum()
totals["Factory"] = "Total"
totals["Warehouse"] = "-"
df_ship1.loc['Total'] = totals

print("\n**Factory -> Warehouse Shipments**:\n")
print(df_ship1)



**Factory -> Warehouse Shipments**:

      Factory Warehouse     P1     P2     P3     P4     P5     P6     P7     P8     P9    P10  Total_Shipment
0          F1        W1   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0             0.0
1          F1        W2   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0             0.0
2          F1        W3  108.0    0.0   56.0    0.0  128.0    0.0  112.0  121.0    0.0    0.0           525.0
3          F1        W4   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0             0.0
4          F1        W5   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0             0.0
5          F2        W1   83.0   50.0   91.0   65.0   70.0   75.0  102.0   93.0   91.0   80.0           800.0
6          F2        W2   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0             0.0
7          F2        W3   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   

##### 3.5. Warehouse to Stores optimal shipping allocation

In [6]:
Ship_WS = []
for k in range(1,6):      # Warehouses
    for s in range(1,6):  # Stores
        row = {
            'Warehouse': f'W{k}',
            'Store': f'S{s}'
        }
        # Har product ka value add
        for i in range(1,11):
            row[f'P{i}'] = value(r[i, k, s])
        Ship_WS.append(row)

df_ship2 = pd.DataFrame(Ship_WS)

# Row total (shipment for each W→S pair across all products)
df_ship2["Total_Shipment"] = df_ship2[[f'P{i}' for i in range(1,11)]].sum(axis=1)

# Column totals (total per product across all routes)
totals = df_ship2.drop(columns=["Warehouse","Store"]).sum()
totals["Warehouse"] = "Total"
totals["Store"] = "-"
df_ship2.loc['Total'] = totals

print("\n**Warehouse -> Store Shipments**:\n")
print(df_ship2)



**Warehouse -> Store Shipments**:

      Warehouse Store     P1     P2     P3     P4     P5     P6     P7     P8     P9    P10  Total_Shipment
0            W1    S1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0             0.0
1            W1    S2    0.0    0.0   48.0   27.0    0.0    0.0   19.0    0.0   43.0   30.0           167.0
2            W1    S3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0             0.0
3            W1    S4   48.0   50.0   43.0   38.0   40.0   45.0   35.0   43.0   48.0   50.0           440.0
4            W1    S5   35.0    0.0    0.0    0.0   30.0   30.0   48.0   50.0    0.0    0.0           193.0
5            W2    S1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0             0.0
6            W2    S2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0             0.0
7            W2    S3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0         